In [145]:
import json
data=json.load(open("2019-05-02-turn1.json"))

In [146]:
data[1]

{'created_at': 'Wed May 01 23:59:59 +0000 2019',
 'id': 1123738872297467904,
 'id_str': '1123738872297467904',
 'text': 'Micro flizz guides data and media through tight spaces https://t.co/NWMgZDzuaj https://t.co/R45xjCuqPN',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/NWMgZDzuaj',
    'expanded_url': 'https://is.gd/TO7y2E',
    'display_url': 'is.gd/TO7y2E',
    'indices': [55, 78]}],
  'media': [{'id': 1123738869869023234,
    'id_str': '1123738869869023234',
    'indices': [79, 102],
    'media_url': 'http://pbs.twimg.com/media/D5hSjF7XkAIDHOc.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/D5hSjF7XkAIDHOc.jpg',
    'url': 'https://t.co/R45xjCuqPN',
    'display_url': 'pic.twitter.com/R45xjCuqPN',
    'expanded_url': 'https://twitter.com/ElectricCXNMag/status/1123738872297467904/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'large': {'w': 600

In [47]:
import json
geo = json.load(open("vicgeo.json")) #VIC Local Government Areas 
VIC_name=[]
for area in geo['features']:
    VIC_name.append(area['properties']['vic_lga__3'])

In [33]:
len(VIC_name)

92

In [48]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [49]:
late_time={'23','00','01','02','03','04','05','06'}

In [50]:
Gluttony_ralated_word=[
'depravity',
'dissipation',
'excess',
'intemperance',
'orgy',
'overeating',
'satiation',
'satiety',
'spoiling',
'overdrinking',
'corpulent',
'obese',
'pudgy',
'fat',
'gross',
'heavy',
'outsize',
'plump',
'stout',
'ample',
'bulky',
'fleshy',
'hefty',
'huge',
'massive',
'overfed',
'overstuffed',
'portly',
'rotund',
'upholstered',
'weighty',
'binge',
'feast',
'gluttonize',
'gorge',
'gourmandize',
'overindulge',
'scarf']

In [118]:
from shapely.geometry import shape, Point
def get_area_name1(geo, coordinate): 
    name=None
    for area in geo['features']:  
        if coordinate.within(shape(area ['geometry'])):
            name = area['properties']['vic_lga__3']
            break
    return name #get area name in VIC by tweet coordinate

def get_area_name2(placename):  
    for item in VIC_name:
        if item in placename.upper():
            return item
            break
    return None #get area name in VIC by tweet place name


In [137]:
import nltk
def Identify_Gluttony(Gluttony_ralated_word,text):#if a tweet capture Gluttony
    Gluttony= 0
    tword = [word.lower() for word in nltk.word_tokenize(text)]
    for term in Gluttony_ralated_word:
         if term in tword :
            Gluttony= 1 # Ture
            break
    return Gluttony
def Identify_late(time,late_time):
    if tweet['user']['created_at'][11:13] in late_time:
            Late_time=1
    else:
            Late_time=0
    return  Late_time     
    

In [140]:
#process a raw tweet to some characteristic label
def process_tweet(tweet,geo,late_time,Gluttony_ralated_word): 
    name=None
    if tweet['coordinates']!=None:
        name=get_area_name1(geo, Point(tweet['coordinates']['coordinates']))
    elif tweet['place']!=None:
        name=get_area_name2(tweet['place']['name'])
    else:
        name=get_area_name2(tweet['user']['location'])
    if name != None:
        
        Late_time=Identify_late(tweet['user']['created_at'][11:13],late_time)
        # 1 when a tweet is in late time
        sentiment_score = analyser.polarity_scores(tweet['text'])['compound']
        #compound score is between -1 and 1
        Gluttony=Identify_Gluttony(Gluttony_ralated_word,tweet['text'])
        # 1 when a tweet contains gluttony words                 
        processed_tweet = {'Vic_area': name,'Late_time':Late_time, 
               'Gluttony':Gluttony,'Sentiment':sentiment_score}
        return processed_tweet
    else:
        return None

In [141]:
result={}
for area in VIC_name:
    result[area]=[0,0,0,0,0.0]

In [144]:
def analysis(processed_tweet,result):
    area=processed_tweet['Vic_area']
    result[area][0]+=1
    result[area][1]+=processed_tweet['Late_time']
    result[area][2]+=processed_tweet['Gluttony']
    if processed_tweet['Sentiment']<0:#calculate the total number of negative tweets
        result[area][3]+=1
    result[area][4]+=processed_tweet['Sentiment']
    